In [106]:
import hashlib # for grading

# Standard imports
import numpy as np
from numpy.testing import assert_allclose
import pandas as pd
import re
import string
import math
import matplotlib.pyplot as plt
import hashlib
import random
import os
from tqdm import tqdm

# NLTK imports
import nltk
nltk.download('stopwords')
from collections import Counter, OrderedDict
from nltk.util import ngrams
import spacy
from spacy.matcher import Matcher

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

# SKLearn related imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from textblob import TextBlob
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings; warnings.simplefilter('ignore')
cpu_count = int(os.cpu_count()) if os.cpu_count() != None else 4

from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rapsag100/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [217]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv("data/test.csv", index_col=0)

In [218]:
train

,Lyrics,Genre
0,[Intro: Method Man w/ sample] + (Sunny valenti...,Hip Hop
1,[Sean Paul:]. Aye. It's Sean Paul 'long side. ...,Pop
2,You've changed your tune. many times since we'...,Rock
3,I got all these J's rolled up. And got all the...,Hip Hop
4,Look I'm standing naked before you. Don't you ...,Rock
...,...,...
39048,It's the end of the movie. And your story's be...,Pop
39049,Uh oh. You live in this lane. Your body's insa...,Hip Hop
39050,Shawty swing my way put that ass all in my fac...,Hip Hop
39051,Born in caught out. Care out fear in. Gear in ...,Rock


In [219]:
train[train.duplicated(keep=False)]

,Lyrics,Genre
312,Summer stretching on the grass. Summer dresses...,Rock
313,No more lying friends. Wanting tragic ends. Th...,Pop
322,French Inhale I like the way you French Inhale...,Hip Hop
618,Yo dreams we all have'em. Wake up in the morni...,Pop
698,All the way in the back with a tree on my lap....,Pop
...,...,...
38459,Maybe it's the way she walked. Straight into m...,Pop
38516,I had so much champagne up in the club. And I ...,Hip Hop
38521,Everynight I go. Everynight I go. Sneaking out...,Pop
38776,Everynight I go. Everynight I go. Sneaking out...,Pop


In [220]:
train[train.Lyrics.duplicated(keep=False)]

,Lyrics,Genre
312,Summer stretching on the grass. Summer dresses...,Rock
313,No more lying friends. Wanting tragic ends. Th...,Pop
322,French Inhale I like the way you French Inhale...,Hip Hop
618,Yo dreams we all have'em. Wake up in the morni...,Pop
698,All the way in the back with a tree on my lap....,Pop
...,...,...
38556,Yeah. Yeah. Haha. It's Travis.. What up!. Bum ...,Hip Hop
38635,Eh whoa yeah yeah yeah. Rihanna. Shy Ronnie. W...,Pop
38776,Everynight I go. Everynight I go. Sneaking out...,Pop
38819,I'm closing all the curtains. So don't you eve...,Rock


In [221]:
train = train.drop_duplicates()
train

,Lyrics,Genre
0,[Intro: Method Man w/ sample] + (Sunny valenti...,Hip Hop
1,[Sean Paul:]. Aye. It's Sean Paul 'long side. ...,Pop
2,You've changed your tune. many times since we'...,Rock
3,I got all these J's rolled up. And got all the...,Hip Hop
4,Look I'm standing naked before you. Don't you ...,Rock
...,...,...
39048,It's the end of the movie. And your story's be...,Pop
39049,Uh oh. You live in this lane. Your body's insa...,Hip Hop
39050,Shawty swing my way put that ass all in my fac...,Hip Hop
39051,Born in caught out. Care out fear in. Gear in ...,Rock


In [222]:
train[train.duplicated(subset=["Lyrics"], keep=False)].to_csv("duplicates.csv")

In [223]:
train = train.drop_duplicates(subset=["Lyrics"])
train

,Lyrics,Genre
0,[Intro: Method Man w/ sample] + (Sunny valenti...,Hip Hop
1,[Sean Paul:]. Aye. It's Sean Paul 'long side. ...,Pop
2,You've changed your tune. many times since we'...,Rock
3,I got all these J's rolled up. And got all the...,Hip Hop
4,Look I'm standing naked before you. Don't you ...,Rock
...,...,...
39048,It's the end of the movie. And your story's be...,Pop
39049,Uh oh. You live in this lane. Your body's insa...,Hip Hop
39050,Shawty swing my way put that ass all in my fac...,Hip Hop
39051,Born in caught out. Care out fear in. Gear in ...,Rock


# Tratamento do texto

In [227]:
train_X = train.drop(columns="Genre")
train_Y = train[["Genre"]]

In [228]:
train_Y

,Genre
0,Hip Hop
1,Pop
2,Rock
3,Hip Hop
4,Rock
...,...
39048,Pop
39049,Hip Hop
39050,Hip Hop
39051,Rock


In [229]:
test

,Lyrics
id,
0,I was nineteen when I came to town they called...
1,Your coat and hat are gone. I've really can't ...
3,I met her at the 'Burger King'. We fell in lov...
5,(Hey Hey). Bye Bye Bye. Bye Bye. Bye Bye. Oh O...
6,One two three four. ONE TWO THREE FOUR. We're ...
...,...
5383,Could you believe. The same old story. It neve...
5384,I can see. by the look on your face. that you'...
5385,I'd move to Rockferry. Tomorrow. And I'd build...


In [230]:
len_test = len(test)
len_test

3354

In [231]:
df = pd.concat([train_X, test])
df

,Lyrics
0,[Intro: Method Man w/ sample] + (Sunny valenti...
1,[Sean Paul:]. Aye. It's Sean Paul 'long side. ...
2,You've changed your tune. many times since we'...
3,I got all these J's rolled up. And got all the...
4,Look I'm standing naked before you. Don't you ...
...,...
5383,Could you believe. The same old story. It neve...
5384,I can see. by the look on your face. that you'...
5385,I'd move to Rockferry. Tomorrow. And I'd build...
5387,Moving on moving on. Isn't that what I'm suppo...


In [232]:
train.Genre.value_counts(normalize=True)

Genre
Rock       0.487121
Pop        0.285450
Hip Hop    0.227429
Name: proportion, dtype: float64

In [233]:
# PLOT BAR 

In [234]:
"""def get_data_stats(X, y):
    print(f"Size of dataset: {len(X)}")
    unique, counts = np.unique(y, return_counts=True)
    print(f"Distribution of classes: {dict(zip(unique, counts))}")
    percent = counts[1] / (counts[1] + counts[0]) 
    print(f"{percent}")
    plt.boxplot(list(zip(unique, counts)), labels=["0", "1"])
    plt.xlabel("Classes")

get_data_stats(X, y)
"""

'def get_data_stats(X, y):\n    print(f"Size of dataset: {len(X)}")\n    unique, counts = np.unique(y, return_counts=True)\n    print(f"Distribution of classes: {dict(zip(unique, counts))}")\n    percent = counts[1] / (counts[1] + counts[0]) \n    print(f"{percent}")\n    plt.boxplot(list(zip(unique, counts)), labels=["0", "1"])\n    plt.xlabel("Classes")\n\nget_data_stats(X, y)\n'

In [235]:
train

,Lyrics,Genre
0,[Intro: Method Man w/ sample] + (Sunny valenti...,Hip Hop
1,[Sean Paul:]. Aye. It's Sean Paul 'long side. ...,Pop
2,You've changed your tune. many times since we'...,Rock
3,I got all these J's rolled up. And got all the...,Hip Hop
4,Look I'm standing naked before you. Don't you ...,Rock
...,...,...
39048,It's the end of the movie. And your story's be...,Pop
39049,Uh oh. You live in this lane. Your body's insa...,Hip Hop
39050,Shawty swing my way put that ass all in my fac...,Hip Hop
39051,Born in caught out. Care out fear in. Gear in ...,Rock


In [236]:
def print_plot(df, index):
    example = df[df.index == index][['Lyrics', 'Genre']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
        
print_plot(train, 0)

[Intro: Method Man w/ sample] + (Sunny valentine). We got butter (8X). (The gun'll go the gun'll go.... The gun'll go...). [Raekwon]. Aiyo one thing for sure keep you of all. Keep a nice crib fly away keep to the point. Keep niggaz outta ya face who snakes. Keep bitches in they place keep the mac in a special place. Keep moving for papes keep cool keep doing what you doing. Keep it fly keep me in the crates. Cuz I will erase shit on the real note you'se a waste. It's right here for you I will lace you. Rip you and brace you put a nice W up on ya face. Word to mother you could get chased. It's nothing to taste blood on a thug if he gotta go. All I know is we be giving grace. This is a place from where we make tapes. We make 'em everywhere still in all we be making base. Y'all be making paste these little niggaz they be making shapes. Our shit is art yours is traced. [Chorus: Sunny Valentine]. This is the way that we rolling in the streets. You know when we roll we be packing that heat. 

In [237]:
print_plot(train, 1)

[Sean Paul:]. Aye. It's Sean Paul 'long side. The mandem called Jay Sean. Fi di gal dem. Tellin' 'em again what we tell 'em. [Jay Sean:]. Pass me a drink to the left yeah. Said her name was Delilah. And I'm like "you should come my way". I already surrender. Damn girl that body's fire. You gon' remember my name. (She should give it up definite). You need it. I need it. We can jump in the deep end. I wanna get lost in your love. I just wanna be close to you. (Just wanna I just wanna). And do all the things you want me to. I just wanna be close to you. (I just wanna I just wanna). And show you the way I feel. You make my love go. You make my love go. You make my love go. In the morning we gon' do it again wake up. I'mma do it like we just broke up and made up. Get up on top of me and work up a sweat work up a sweat. See we can do it any type of way that you want. I'm thinking maybe you're the right kind of wrong. I'm saying baby you won't ever forget my love. You need it. I need it. We c

In [238]:
print_plot(train, 28)

Nosin' your blow. in the wintertime baby. nosin' your blow.. Nosin' your blow baby.. And you're crazy.. You go crazy.. You're going in-sane!!!. Baby. don't worry about it.. Have another sandwich.. Nosin' your blow.. Got nowhere to go. in the snow..
Tag: Rock


In [239]:
STOPWORDS = set(nltk.corpus.stopwords.words('english'))
#BAD_SYMBOLS_RE = re.compile('[^0-9a-z ]')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# re.compile("[" + re.escape(string.punctuation)+ "]")
#pattern_words = r"
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    words = WordPunctTokenizer().tokenize(text)
    res = " ".join(words)
    res = res.lower() # lowercase text
    res = re.sub(r'[^a-zA-Z0-9\s]', '', res)
    #res = REPLACE_BY_SPACE_RE.sub(' ', res) 
    #text = BAD_SYMBOLS_RE.sub('', text)
    stemmer = SnowballStemmer("english")
    words = res.split()
    stems = [stemmer.stem(word) for word in words]
    sentence_stemmed = " ".join(stems)
    text = ' '.join(word for word in sentence_stemmed.split() if word not in STOPWORDS)
    text = re.sub(r"^\s+|\s+$|(?<=\s)\s*", "", text)
    return text


In [240]:
df_proc = df.copy()
df_proc['Lyrics'] =df_proc['Lyrics'].apply(clean_text)

In [241]:
def print_plot_proc(df, index):
    example = df[df.index == index][['Lyrics']].values[0]
    if len(example) > 0:
        print(example[0])
        #print('Tag:', example[1])
        
print_plot_proc(df_proc, 0)

intro method man w sampl sunni valentin got butter 8x gun go gun go gun go raekwon aiyo one thing sure keep keep nice crib fli away keep point keep niggaz outta ya face snake keep bitch place keep mac special place keep move pape keep cool keep keep fli keep crate cuz eras shit real note se wast right lace rip brace put nice w ya face word mother could get chase noth tast blood thug gotta go know give grace place make tape make em everywher still make base make past littl niggaz make shape shit art trace chorus sunni valentin way roll street know roll pack heat gun go gun go gun go gun go gun go gun go gun go gun go gun go gun go method man poverti island man anim run slum ambul come got best base fiend wait smoke approach someth ask get coke dude hug block like samurai shogun cuz v one equal fun want treat know huh bodi go numb woman need fund plus hair toe done though fuck kid flow make hard get dough harder get gold harder piff blow harder snow pinki wrist glow live get gwop get low

In [242]:
print_plot_proc(df_proc, 1)

sean paul aye sean paul long side mandem call jay sean fi di gal dem tellin em tell em jay sean pass drink left yeah said name delilah like come way alreadi surrend damn girl bodi fire gon rememb name give definit need need jump deep end wanna get lost love wanna close wanna wanna thing want wanna close wanna wanna show way feel make love go make love go make love go morn gon wake mma like broke made get top work sweat work sweat see ani type way want think mayb right kind wrong say babi ever forget love need need jump deep end wanna get lost love wanna close wanna wanna thing want wanna close wanna wanna show way feel make love go make love go make love go sean paul girl mi wan figur hundr hundr fifti love move know perfect size know fit let hit know mi quit pon di dl like cassi diddi mi na wound mi watch like sin citi full time mi run da ting mi tall legend come gimm dat would offend girl come wan see someth want life wast time mi pree everi day babi full time ya de pon mi mind mi wi

In [243]:
test_X_proc = df_proc[-len_test:]
test_X_proc

,Lyrics
0,nineteen came town call summer love burn babi ...
1,coat hat gone realli look littl empti shelf ra...
3,met burger king fell love soda machin took car...
5,hey hey bye bye bye bye bye bye bye oh oh doin...
6,one two three four one two three four chipmunk...
...,...
5383,could believ old stori never bore though heard...
5384,see look face got ringworm veri sorri tell got...
5385,move rockferri tomorrow build hous babi sorrow...
5387,move move suppos hold back keep move push push...


In [244]:
train_X_proc = df_proc[:-len_test]
train_X_proc

,Lyrics
0,intro method man w sampl sunni valentin got bu...
1,sean paul aye sean paul long side mandem call ...
2,chang tune mani time sinc met alway recogn par...
3,got j roll got drink pour bought bottl club ca...
4,look stand nake befor want sex scream loud las...
...,...
39048,end movi stori told one unhappi one got old he...
39049,uh oh live lane bodi insan wanna motion like o...
39050,shawti swing way put ass face round round abov...
39051,born caught care fear gear heart mean fear war...


# Analise de dados

In [245]:
def extract_features_chi2(X_train, y_train, top_features=100):
    vectorizer = TfidfVectorizer(min_df=4)
    vectorizer.fit(X_train)
    vec_X_train = vectorizer.transform(X_train)
    feature_names = vectorizer.get_feature_names_out()
    
    ch2 = SelectKBest(chi2, k=top_features)
    ch2.fit(vec_X_train, y_train)
    X_train_ch2 = ch2.transform(vec_X_train)
    
    feat_index = ch2.get_support(indices=True)
    feat_score = ch2.scores_[feat_index]
    dict_feats = dict(zip(feat_index, feat_score))
    dict_names = [(feature_names[idx], value) for idx, value in dict_feats.items()]
    ngrams_sorted = sorted(dict_names, key=lambda x: -x[1])
    return [w for w, i in ngrams_sorted]


In [246]:
most_important_features = extract_features_chi2(train_X_proc.Lyrics, train_Y, top_features=100)
most_important_features

['nigga',
 'bitch',
 'shit',
 'fuck',
 'niggaz',
 'yo',
 'em',
 'ya',
 'ass',
 'vers',
 'hoe',
 'gon',
 'wit',
 'hook',
 'money',
 'fuckin',
 'motherfuck',
 'rap',
 '50',
 'lil',
 'hood',
 'like',
 'pussi',
 'dogg',
 'homi',
 'gangsta',
 'uh',
 'tryna',
 'dick',
 'snoop',
 'thug',
 'aint',
 'im',
 'pop',
 'get',
 'bout',
 'hit',
 'game',
 'rapper',
 'got',
 'girl',
 'cent',
 'club',
 'shawti',
 'thang',
 'gettin',
 'tha',
 'weed',
 'block',
 'mma',
 'huh',
 'motherfuckin',
 'dat',
 'eminem',
 'jay',
 'dope',
 'cuz',
 'smoke',
 'ghetto',
 'dem',
 'uhh',
 'pimp',
 'shorti',
 'gucci',
 'real',
 'fo',
 'damn',
 'crib',
 'chick',
 'bust',
 'hater',
 'cube',
 'dre',
 'drop',
 '40',
 'cop',
 'chorus',
 'love',
 'tryin',
 'wanna',
 'poppin',
 'plus',
 'mo',
 'hustl',
 'intro',
 'beef',
 'yea',
 'shot',
 'cash',
 'check',
 'man',
 'put',
 'caus',
 'blunt',
 'mic',
 'talkin',
 'murder',
 'glock',
 'holla',
 'oh']

In [247]:
train_analy = pd.concat([train_X_proc, train_Y], axis=1)

for feature in most_important_features:
    print(f"for: {feature}")
    print("----")
    for genre in ["Hip Hop", "Pop", "Rock"]:
        genre_texts = train_analy[train_analy.Genre == genre].Lyrics.to_list()
        n = len([word for text in genre_texts for word in text.split() if word == feature])
        print(f"{genre} has: {n} cases")
    print("\n")

for: nigga
----
Hip Hop has: 28915 cases
Pop has: 341 cases
Rock has: 56 cases


for: bitch
----
Hip Hop has: 14328 cases
Pop has: 830 cases
Rock has: 453 cases


for: shit
----
Hip Hop has: 14795 cases
Pop has: 821 cases
Rock has: 779 cases


for: fuck
----
Hip Hop has: 14340 cases
Pop has: 944 cases
Rock has: 2046 cases


for: niggaz
----
Hip Hop has: 4903 cases
Pop has: 31 cases
Rock has: 10 cases


for: yo
----
Hip Hop has: 7459 cases
Pop has: 761 cases
Rock has: 341 cases


for: em
----
Hip Hop has: 9985 cases
Pop has: 1109 cases
Rock has: 1584 cases


for: ya
----
Hip Hop has: 13091 cases
Pop has: 3236 cases
Rock has: 2739 cases


for: ass
----
Hip Hop has: 6671 cases
Pop has: 417 cases
Rock has: 486 cases


for: vers
----
Hip Hop has: 6544 cases
Pop has: 693 cases
Rock has: 466 cases


for: hoe
----
Hip Hop has: 4134 cases
Pop has: 132 cases
Rock has: 23 cases


for: gon
----
Hip Hop has: 5183 cases
Pop has: 880 cases
Rock has: 77 cases


for: wit
----
Hip Hop has: 4636 cases
Po

# Feat eng on train and val

In [249]:
class Selector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a column from the dataframe to perform additional transformations on
    """ 
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self

class TextSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def transform(self, X):
        return X[self.key]

class NumberSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def transform(self, X):
        return X[[self.key]]

In [250]:
def words_nmb(text):
    words = text.split()
    return len(words)

def count_swear_words(text):
    swear_words = ["shit", "bitch", "asshole", "dick", "cunt", "bastard",
    "motherfuck", "cock", "piss", "twat", "ass", "damn", "hell",
    "bollock", "arsehol", "wanker", "prick", "slut", "whore", "mufucka", "fuckin"] 
    swear_word_count = len([word for word in text.split() if word in swear_words])
    return swear_word_count

def count_pop_words(text):
    pop_words = ['love', 'kiss', 'baby', 'danc', 'oh', "pop", "nigga", "hoe", "gon", "tryna", "shorti",
                "shawti", "mma", "wanna"]
    pop_words_count = len([word for word in text.split() if word in pop_words])
    return pop_words_count

def count_rock_words(text):
    rock_words = ["fuck", "fucking", "motherfuckin", "tha"]
    pop_words_count = len([word for word in text.split() if word in rock_words])
    return pop_words_count

def sentiment_score_extractor(text):
    return TextBlob(text).sentiment.polarity


In [251]:
def pre_process(df_):
    df_processed = df_.copy()
    df_processed["wrd_nmb"] = df_processed["Lyrics"].apply(words_nmb)
    df_processed['swear_words_count'] = df_processed['Lyrics'].apply(count_swear_words)
    
    df_processed['pop_words_count'] = df_processed['Lyrics'].apply(count_pop_words)
    df_processed['sentiment_score'] = df_processed['Lyrics'].apply(sentiment_score_extractor)
    
    df_processed['rock_words'] = df_processed['Lyrics'].apply(count_rock_words)
        

    return df_processed

# Predictions on validation

In [253]:
X_train, X_val, y_train, y_val = train_test_split(train_X_proc, train_Y, random_state=42, test_size=0.1, stratify=train_Y.Genre)
# por os dados processados (lower, punct, stem, token)
X_train

,Lyrics
13625,sh sh shut shut fuck hello get shut fuck guy g...
24730,bun b vers 1 come make money print press come ...
18100,found thrill blueberri hill blueberri hill fou...
6912,tri pretend happen like never thought would en...
23656,far side wish electr sky want catch fire find ...
...,...
25862,black come back pretend make home everyth red ...
1700,drive pretti car know fine nobodi need say way...
31058,said let get town drive citi away crowd though...
15580,grew mississippi delta shotgun shack washboard...


In [259]:
df_train_processed = pre_process(X_train)
df_val_processed = pre_process(X_val)


In [260]:
text_pipe = Pipeline([('selector', TextSelector(key='Lyrics')),
                      ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=4))])

word_nmb_pipe = Pipeline([('selector', NumberSelector(key="wrd_nmb")),
                          ('scaler', StandardScaler())])

swear_words_count_pipe = Pipeline([('selector', NumberSelector(key='swear_words_count')),
                                   ('scaler', StandardScaler())])

pop_words_count_pipe = Pipeline([('selector', NumberSelector(key='pop_words_count')),
                                 ('scaler', StandardScaler())])

sentiment_score_pipe = Pipeline([('selector', NumberSelector(key='sentiment_score')),
                                 ('scaler', StandardScaler())])

rock_words_pipe = Pipeline([('selector', NumberSelector(key='rock_words')),
                            ('scaler', StandardScaler())])

feats = FeatureUnion([('Lyrics', text_pipe),
                      ('nb_words', word_nmb_pipe),
                      ("rock", rock_words_pipe),
                      ('swear_words_count', swear_words_count_pipe),
                      ('pop_word_count', pop_words_count_pipe),
                      ('sentiment_score', sentiment_score_pipe)])

In [261]:
label_encoder = LabelEncoder()

# Fit the encoder to your labels to establish the mapping
label_encoder.fit(train_Y)

# Transform labels to integers
y_train_encoded = label_encoder.transform(train_Y)
y_train_encoded

array([0, 1, 2, ..., 0, 2, 1])

In [262]:
my_tags = ["Hip Hop", "Pop", "Rock"]

In [264]:
label_encoder = LabelEncoder()

# Fit the encoder to your labels to establish the mapping
label_encoder.fit(train_Y)

# Transform labels to integers
y_train_encoded = label_encoder.transform(train_Y)
y_train_encoded

array([0, 1, 2, ..., 0, 2, 1])

In [265]:
pipe = Pipeline([('feats', feats),
                 ('clf', LogisticRegression())])

pipe.fit(df_train_processed, y_train)
y_pred = pipe.predict(df_val_processed)
#y_pred.map({0: 'Hip Hop', 1: 'Pop', 2: 'Rock'})
print('f1_score %s' % f1_score(y_pred, y_val, average='weighted'))
print(classification_report(y_val, y_pred, target_names=my_tags))

f1_score 0.7488132662786129
              precision    recall  f1-score   support

     Hip Hop       0.90      0.77      0.83       885
         Pop       0.62      0.50      0.55      1110
        Rock       0.74      0.87      0.80      1895

    accuracy                           0.74      3890
   macro avg       0.75      0.71      0.73      3890
weighted avg       0.74      0.74      0.74      3890



# Doc2vec (good score but unable to mix with other feats)

In [28]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled


In [29]:
X_train_lab = label_sentences(X_train, 'Train')
X_val_lab = label_sentences(X_val, 'Val')

In [30]:
all_data = X_train_lab + X_val_lab
all_data[:2]

[TaggedDocument(words=['must', 'drank', 'lot', 'pain', 'judg', 'way', 'ach', 'must', 'sold', 'lot', 'secret', 'everyon', 'heavi', 'weight', 'feel', 'fade', 'first', 'one', 'defeat', 'everyth', 'know', 'everyth', 'took', 'em', 'shook', 'em', 'floor', 'laid', 'ghost', 'hate', 'blew', 'away', 'close', 'door', 'heal', 'love', 'like', 'love', 'like', 'yeah', 'heal', 'love', 'like', 'love', 'like', 'love', 'bitter', 'tast', 'lie', 'wake', 'least', 'know', 'got', 'reason', 'mother', 'relat', 'better', 'place', 'know', 'broken', 'even', 'everyth', 'know', 'everyth', 'took', 'em', 'shook', 'em', 'floor', 'laid', 'ghost', 'hate', 'blew', 'away', 'close', 'door', 'heal', 'love', 'like', 'love', 'like', 'yeah', 'heal', 'love', 'like', 'love', 'like', 'nobodi', 'know', 'like', 'babi', 'know', 'nobodi', 'goe', 'like', 'babi', 'goe', 'nobodi', 'know', 'like', 'babi', 'know', 'nobodi', 'nobodi', 'heal', 'love', 'like', 'love', 'like', 'yeah', 'heal', 'love', 'like', 'love', 'like', 'heal', 'love', 'li

In [31]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=3, alpha=0.05)
model_dbow.build_vocab([x for x in tqdm(all_data)])

model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=10)

100%|█████████████████████████████████████████████████████████████████████| 38900/38900 [00:00<00:00, 5564558.70it/s]


In [32]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors


In [33]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train_lab), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_val_lab), 300, 'Val')
train_vectors_dbow.shape[0]

31120

In [34]:
my_tags = ["Hip Hop", "Pop", "Rock"]
logreg = LogisticRegression(random_state=42)
logreg.fit(train_vectors_dbow, y_train)
#logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('f1_score %s' % f1_score(y_pred, y_val, average='weighted'))
print(classification_report(y_val, y_pred, target_names=my_tags))

f1_score 0.7388833630466843
              precision    recall  f1-score   support

     Hip Hop       0.86      0.81      0.83      1769
         Pop       0.60      0.47      0.53      2221
        Rock       0.74      0.85      0.79      3790

    accuracy                           0.73      7780
   macro avg       0.73      0.71      0.72      7780
weighted avg       0.72      0.73      0.72      7780



# Doc2vec (test) but doesnt work well

In [35]:
train_X_proc

,Lyrics
0,intro method man w sampl sunni valentin got bu...
1,sean paul aye sean paul long side mandem call ...
2,chang tune mani time sinc met alway recogn par...
3,got j roll got drink pour bought bottl club ca...
4,look stand nake befor want sex scream loud las...
...,...
39048,end movi stori told one unhappi one got old he...
39049,uh oh live lane bodi insan wanna motion like o...
39050,shawti swing way put ass face round round abov...
39051,born caught care fear gear heart mean fear war...


In [36]:
test_X_proc

,Lyrics
0,nineteen came town call summer love burn babi ...
1,coat hat gone realli look littl empti shelf ra...
3,met burger king fell love soda machin took car...
5,hey hey bye bye bye bye bye bye bye oh oh doin...
6,one two three four one two three four chipmunk...
...,...
5383,could believ old stori never bore though heard...
5384,see look face got ringworm veri sorri tell got...
5385,move rockferri tomorrow build hous babi sorrow...
5387,move move suppos hold back keep move push push...


In [37]:
all_data_proc = pd.concat([train_X_proc, test_X_proc])
all_data_proc

,Lyrics
0,intro method man w sampl sunni valentin got bu...
1,sean paul aye sean paul long side mandem call ...
2,chang tune mani time sinc met alway recogn par...
3,got j roll got drink pour bought bottl club ca...
4,look stand nake befor want sex scream loud las...
...,...
5383,could believ old stori never bore though heard...
5384,see look face got ringworm veri sorri tell got...
5385,move rockferri tomorrow build hous babi sorrow...
5387,move move suppos hold back keep move push push...


In [38]:
X_traint_lab = label_sentences(train_X_proc.Lyrics, 'Train')
X_test_lab = label_sentences(test_X_proc.Lyrics, 'Test')

In [39]:
all_data = X_traint_lab + X_test_lab

In [40]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=4, min_count=4, alpha=0.05)
model_dbow.build_vocab([x for x in tqdm(all_data)])

model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=10)

100%|█████████████████████████████████████████████████████████████████████| 42254/42254 [00:00<00:00, 4899403.46it/s]


In [41]:
train_vectors_dbow = get_vectors(model_dbow, len(X_traint_lab), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test_lab), 300, 'Test')
train_vectors_dbow.shape[0]

38900

In [42]:
train_vectorize_itai = pd.DataFrame(train_vectors_dbow)
train_vectorize_itai

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.376665,0.424109,-0.558775,-0.052983,-0.059096,0.538325,0.210362,0.037822,0.117256,0.178907,...,0.174765,-0.023983,-0.017750,-0.344989,0.480928,-0.286509,0.392130,-0.353823,0.230292,0.082696
1,-0.457368,0.484606,-0.115874,0.103886,0.093170,-0.080585,0.160376,0.165359,-0.097038,0.025924,...,0.113422,0.624249,-0.179477,0.100905,-0.263187,-0.087620,-0.162521,-0.002672,0.205060,-0.237375
2,-0.008527,0.164607,-0.142613,-0.520444,0.207883,0.245330,-0.221399,0.162824,0.070214,0.329242,...,0.454509,0.060622,0.715117,0.292663,0.071411,0.079499,-0.194757,-0.586553,0.051116,0.351184
3,-0.257963,0.553346,-0.463751,-0.189711,-0.202404,-0.125804,-0.250566,0.510760,0.036228,-0.212939,...,-0.250400,0.423471,-0.106641,0.253176,0.053248,-0.255988,0.008130,-0.138190,0.030266,0.065772
4,0.237859,-0.146387,0.126947,-0.200516,-0.018876,0.167701,0.217484,-0.059748,-0.358885,-0.342817,...,-0.533138,0.353774,-0.238853,0.137210,-0.023942,0.283200,0.381714,-0.108016,0.189693,-0.053854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38895,-0.257365,-0.183011,-0.121573,0.067782,-0.218551,-0.202579,-0.157247,0.236703,-0.257715,-0.070360,...,0.289090,0.839509,-0.176878,0.193810,0.288844,-0.282178,-0.167662,0.302356,0.016821,-0.173997
38896,-0.165347,0.128604,0.182705,-0.018585,-0.116201,-0.150407,0.279838,0.388983,-0.359551,-0.267743,...,0.380250,0.530753,0.136435,-0.714736,-0.098237,0.066060,-0.204750,-0.493877,0.076196,-0.013987
38897,0.193818,0.497728,0.169042,-0.073872,-0.227782,0.201108,-0.205055,0.138676,0.281999,-0.231063,...,-0.076051,0.114871,-0.303503,-0.443079,0.196701,0.072180,-0.027419,-0.440790,0.137578,0.289068
38898,-0.234026,-0.284741,-0.790537,-0.096799,-0.061614,0.314586,0.083052,0.164028,-0.378573,-0.121432,...,-0.480044,-0.115849,-0.034477,0.025676,0.138705,0.025958,0.207007,0.267025,0.110835,0.236255


In [43]:
test_vectorize_itai = pd.DataFrame(test_vectors_dbow)
test_vectorize_itai

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.488700,-0.256337,-0.437417,-0.197451,-0.619259,0.222918,0.072599,0.385336,0.025140,-0.699467,...,-0.291687,-0.046030,0.092195,0.224681,-0.432090,-0.093882,-0.266385,-0.368588,-0.141070,0.318756
1,-0.118392,0.014933,-0.346994,0.016643,0.274571,0.418750,-0.373841,0.033258,-0.617707,0.090672,...,-0.132286,0.085536,-0.647508,0.045748,-0.360975,0.115141,-0.019637,0.124606,-0.040296,-0.089911
2,-0.172584,-0.364350,0.012121,-0.549759,0.371595,-0.031221,0.006977,-0.110576,0.043514,0.414844,...,0.267099,0.681907,-0.229980,-0.139553,-0.488318,-0.148865,-0.095369,0.077446,-0.188926,0.032231
3,-0.206126,0.031731,0.037615,-0.076575,-0.277545,-0.233811,-0.099873,-0.328007,-0.716818,-0.301505,...,0.487848,0.323571,0.180602,-0.397048,0.021972,-0.354067,-0.494979,0.040916,0.262422,0.252070
4,0.001027,-0.166254,-0.084452,0.099946,-0.553498,-0.090856,0.120897,-0.136964,0.064928,-0.130858,...,0.105283,0.335415,0.031874,-0.232854,-0.209804,-0.370405,-0.041949,0.152636,0.283983,0.084365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3349,-0.128443,0.516778,-0.009152,-0.059018,-0.821634,-0.166702,-0.185200,0.016248,0.107155,-0.118170,...,0.259549,0.229729,0.123338,-0.376780,-0.463868,-0.270834,-0.563015,-0.430960,-0.236684,0.393259
3350,-0.144446,-0.052555,-0.012802,0.028669,0.102317,-0.289133,-0.055571,0.172506,0.003524,0.047743,...,0.474129,0.107089,-0.235107,0.099656,0.204516,-0.143147,-0.129425,0.255445,-0.192629,-0.072452
3351,0.007985,-0.080680,-0.132639,-0.086080,0.017598,0.222055,0.016095,-0.124093,-0.160533,0.293298,...,-0.000710,-0.141005,0.254934,-0.376417,-0.094837,0.017793,-0.351268,0.152541,-0.099861,-0.257837
3352,-0.220813,0.308580,-0.477849,-0.152009,-0.402102,0.028367,0.300271,-0.210788,-0.196248,0.155102,...,0.114178,0.195180,0.726438,-0.008342,0.236520,0.152669,-0.077273,-0.182194,0.283631,0.123768


In [44]:
"""train_vectorize_itai.to_csv("train_vectorized.csv")
test_vectorize_itai.to_csv("test_vectorized.csv")
"""

'train_vectorize_itai.to_csv("train_vectorized.csv")\ntest_vectorize_itai.to_csv("test_vectorized.csv")\n'

# Feature Eng

In [187]:
df_train_processed = pre_process(train_X_proc)
df_test_processed = pre_process(test_X_proc)


In [196]:
df_train_processed = df_train_processed.reset_index().drop(columns="index")


,Lyrics,wrd_nmb,swear_words_count,pop_words_count,sentiment_score,rock_words
0,intro method man w sampl sunni valentin got bu...,337,4,0,0.046214,2
1,sean paul aye sean paul long side mandem call ...,237,1,36,0.334656,0
2,chang tune mani time sinc met alway recogn par...,141,0,0,0.190000,0
3,got j roll got drink pour bought bottl club ca...,210,1,12,0.021984,0
4,look stand nake befor want sex scream loud las...,87,0,14,0.376923,0
...,...,...,...,...,...,...
38895,end movi stori told one unhappi one got old he...,170,0,1,-0.042807,0
38896,uh oh live lane bodi insan wanna motion like o...,259,2,21,0.074242,0
38897,shawti swing way put ass face round round abov...,185,3,8,0.133333,0
38898,born caught care fear gear heart mean fear war...,312,4,4,-0.051447,0


In [197]:
df_train_processed

,Lyrics,wrd_nmb,swear_words_count,pop_words_count,sentiment_score,rock_words
0,intro method man w sampl sunni valentin got bu...,337,4,0,0.046214,2
1,sean paul aye sean paul long side mandem call ...,237,1,36,0.334656,0
2,chang tune mani time sinc met alway recogn par...,141,0,0,0.190000,0
3,got j roll got drink pour bought bottl club ca...,210,1,12,0.021984,0
4,look stand nake befor want sex scream loud las...,87,0,14,0.376923,0
...,...,...,...,...,...,...
38895,end movi stori told one unhappi one got old he...,170,0,1,-0.042807,0
38896,uh oh live lane bodi insan wanna motion like o...,259,2,21,0.074242,0
38897,shawti swing way put ass face round round abov...,185,3,8,0.133333,0
38898,born caught care fear gear heart mean fear war...,312,4,4,-0.051447,0


# Analise dos dados

In [69]:
train_analy = pd.concat([df_train_processed, train_Y.reset_index().drop(columns="index")], axis=1)
train_analy

,Lyrics,wrd_nmb,doc_length,swear_words_count,pop_words_count,sentiment_score,unique_word_density,Genre
0,intro method man w sampl sunni valentin got bu...,337,1700,6,0,0.046214,0.623145,Hip Hop
1,sean paul aye sean paul long side mandem call ...,237,1168,1,13,0.334656,0.489451,Pop
2,chang tune mani time sinc met alway recogn par...,141,795,0,0,0.190000,0.475177,Rock
3,got j roll got drink pour bought bottl club ca...,210,1114,2,1,0.021984,0.476190,Hip Hop
4,look stand nake befor want sex scream loud las...,87,458,0,14,0.376923,0.505747,Rock
...,...,...,...,...,...,...,...,...
38895,end movi stori told one unhappi one got old he...,170,988,0,0,-0.042807,0.429412,Pop
38896,uh oh live lane bodi insan wanna motion like o...,259,1075,2,10,0.074242,0.355212,Hip Hop
38897,shawti swing way put ass face round round abov...,185,951,3,1,0.133333,0.405405,Hip Hop
38898,born caught care fear gear heart mean fear war...,312,1710,4,4,-0.051447,0.733974,Rock


In [74]:
train_analy.groupby("Genre")["sentiment_score"].mean()

Genre
Hip Hop    0.056964
Pop        0.140930
Rock       0.103614
Name: sentiment_score, dtype: float64

## NOTA

A junção de Doc2vec e feats dá um pior resultado no test 60% ... com LR

In [51]:
#train_vec_feat = pd.concat([df_train_processed, train_vectorize_itai], axis=1)
#train_vec_feat

In [53]:
#test_vec_feat = pd.concat([df_test_processed, test_vectorize_itai], axis=1)
#test_vec_feat

In [54]:
#train_vec_feat.columns = train_vec_feat.columns.astype(str)
#test_vec_feat.columns = test_vec_feat.columns.astype(str)


In [55]:
#train_vec_feat = train_vec_feat.drop(columns=["Lyrics"])
#test_vec_feat = test_vec_feat.drop(columns=["Lyrics"])


In [56]:
#train_vec_feat

# Predictions on test

In [189]:
text_pipe = Pipeline([('selector', TextSelector(key='Lyrics')),
                      ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=4))])

word_nmb_pipe = Pipeline([('selector', NumberSelector(key="wrd_nmb")),
                          ('scaler', StandardScaler())])

swear_words_count_pipe = Pipeline([('selector', NumberSelector(key='swear_words_count')),
                                   ('scaler', StandardScaler())])

pop_words_count_pipe = Pipeline([('selector', NumberSelector(key='pop_words_count')),
                                 ('scaler', StandardScaler())])

sentiment_score_pipe = Pipeline([('selector', NumberSelector(key='sentiment_score')),
                                 ('scaler', StandardScaler())])

rock_words_pipe = Pipeline([('selector', NumberSelector(key='rock_words')),
                            ('scaler', StandardScaler())])

feats = FeatureUnion([('Lyrics', text_pipe),
                      ('nb_words', word_nmb_pipe),
                      ("rock", rock_words_pipe),
                      ('swear_words_count', swear_words_count_pipe),
                      ('pop_word_count', pop_words_count_pipe),
                      ('sentiment_score', sentiment_score_pipe)])

In [ ]:
label_encoder = LabelEncoder()

# Fit the encoder to your labels to establish the mapping
label_encoder.fit(train_Y)

# Transform labels to integers
y_train_encoded = label_encoder.transform(train_Y)
y_train_encoded

In [212]:
pipe = Pipeline([('feats', feats),
                 ('clf', LogisticRegression())])

pipe.fit(df_train_processed, y_train_encoded)
y_pred = pipe.predict(df_test_processed)


In [213]:
"""#my_tags = ["Hip Hop", "Pop", "Rock"]
logreg = LogisticRegression(random_state=42)
logreg.fit(train_vec_feat, train_Y.Genre)
y_pred = logreg.predict(test_vec_feat)
#print('f1_score %s' % f1_score(y_pred, y_val, average='weighted'))
#print(classification_report(y_val, y_pred, target_names=my_tags))
"""

'#my_tags = ["Hip Hop", "Pop", "Rock"]\nlogreg = LogisticRegression(random_state=42)\nlogreg.fit(train_vec_feat, train_Y.Genre)\n#logreg = logreg.fit(train_vectors_dbow, y_train)\ny_pred = logreg.predict(test_vec_feat)\n#print(\'f1_score %s\' % f1_score(y_pred, y_val, average=\'weighted\'))\n#print(classification_report(y_val, y_pred, target_names=my_tags))\n'

In [214]:
final_fd = pd.DataFrame(y_pred, columns=["Genre"])["Genre"].map({0: 'Hip Hop', 1: 'Pop', 2: 'Rock'})
df_sub = pd.DataFrame(final_fd, columns=["Genre"])
df_sub

,Genre
0,Rock
1,Rock
2,Rock
3,Pop
4,Pop
...,...
3349,Pop
3350,Rock
3351,Rock
3352,Rock


In [215]:
df_sub.to_csv("submission.csv", index=False)